In [ ]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from src.data_loader import get_data_generators
from src.model import build_model

In [ ]:
DATA_DIR = '../data/raw'  
IMG_SIZE = (224, 224)     
BATCH_SIZE = 32           
EPOCHS = 30               

# Check if data directory exists
if not os.path.exists(DATA_DIR):
    print(f"Error: {DATA_DIR} not found. Please make sure you have downloaded the data!")
else:
    print(f"Data folder found: {DATA_DIR}")

In [ ]:
# Data Generators
print("\n--- Data loading ---")
try:
    train_gen, val_gen = get_data_generators(DATA_DIR, img_size=IMG_SIZE, batch_size=BATCH_SIZE)
    print(f"Found classes: {train_gen.num_classes}")
    print(f"Class names: {list(train_gen.class_indices.keys())}")
except Exception as e:
    print(f"Error loading data: {e}")

In [ ]:
# Build Model
print("\n--- Building model MobileNetV2 ---")
num_classes = train_gen.num_classes
model = build_model(num_classes=num_classes, input_shape=IMG_SIZE + (3,))
model.summary()

In [ ]:
# Fine tuning setup
model.trainable = True

for layer in model.layers[:100]:
    layer.trainable = False


model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


print(f"Number of frozen layers: 100")
print(f"Number of trainable layers: {len(model.layers) - 100}")


In [ ]:
print(f"Model gotowy do Fine-Tuningu. Liczba warstw do treningu: {len(model.layers) - 100}")

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_lr=1e-6), 
    ModelCheckpoint('../models/cloud_model_best.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

In [ ]:
# Training
print(f"\n--- Training ({EPOCHS} epochs) ---")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

print("\n--- Training complete ---")

In [ ]:
# Results 
print("\n--- Saving model ---")
if not os.path.exists('../models'):
    os.makedirs('../models')

save_path = '../models/cloud_model_best.h5'
model.save(save_path)
print(f"SUCCESS! Model saved at: {save_path}")

✅ Folder danych znaleziony: ../data/raw

--- Przygotowywanie generatorów danych ---
Found 2039 images belonging to 11 classes.
Found 504 images belonging to 11 classes.
Znaleziono klas: 11
Nazwy klas: ['Ac', 'As', 'Cb', 'Cc', 'Ci', 'Cs', 'Ct', 'Cu', 'Ns', 'Sc', 'St']

--- Pobieranie modelu MobileNetV2 ---
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

--- Start treningu (5 epok) ---
Epoch 1/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 447s 7s/step - accuracy: 0.3001 - loss: 2.0253 - val_accuracy: 0.3254 - val_loss: 1.9449
Epoch 2/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 13s 196ms/step - accuracy: 0.4340 - loss: 1.6671 - val_accuracy: 0.3353 - val_loss: 1.9358
Epoch 3/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 13s 204ms/step - accuracy: 0.4718 - loss: 1.5559 - val_accuracy: 0.3512 - val_loss: 1.9221
Epoch 4/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 13s 208ms/step - accuracy: 0.4895 - loss: 1.4682 - val_accuracy: 0.3492 - val_loss: 1.8957
Epoch 5/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 13s 208ms/step - accuracy: 0.5091 - loss: 1.4271 - val_accuracy:


--- Zapisywanie modelu ---
✅ SUKCES! Model zapisany w: ../models/cloud_model_v1.h5
